# <font color="purple">Adversarial Training of Residual Network</font>

**Libraries**

In [ ]:
#!pip install -qq -e git+http://github.com/tensorflow/cleverhans.git#egg=cleverhans
import sys
sys.path.append('/content/src/cleverhans')

import cleverhans

In [ ]:
!pip install hickle

In [ ]:
import hickle as hkl

In [ ]:
import tensorflow as tf
from cleverhans.future.tf2.attacks import fast_gradient_method
from tensorflow.keras.optimizers import SGD
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from sklearn.model_selection import  KFold
from wresnet import WideResidualNetwork
import warnings
warnings.filterwarnings("ignore")

print("\nTensorflow Version: " + tf.__version__)

from parsevalnet import ParsevalNetwork

from _utility import lrate, get_adversarial_examples, print_test, step_decay

## <font color="green"> Data Preprocessing </font>


* Read Data from File

In [ ]:
data = hkl.load("data.hkl")
X_train, X_test, Y_train, y_test = data['xtrain'], data['xtest'], data['ytrain'], data['ytest']

### <font color = "green">Some Parameters </font>

In [ ]:
epsilons = [i/1000 for i in range(1,33)] # facto for fast gradient sign method

In [ ]:
EPOCHS = 50
BS = 64
init = (32, 32,1)
sgd = SGD(lr=0.1, momentum=0.9)
parameter = {'epochs': EPOCHS, 'batch_size': BS, 'optimizer': sgd}

In [ ]:
wideresnet = WideResidualNetwork(init, 0.0001, 0.9, nb_classes=4, N=2, k=1, dropout=0.0)

In [ ]:
from adversarial_training import AdversarialTraining

In [ ]:
adversarial_training = AdversarialTraining(parameter)

In [ ]:
def simulate_train(s):
    
    kfold = KFold(n_splits=10, random_state=42, shuffle=False)
    model_name="ResNet_da"

    for j, (train, val) in enumerate(kfold.split(X_train)):
        if j == s :
            print(s)
            model = wideresnet.create_wide_residual_network()
            model.compile(loss="categorical_crossentropy",
                        optimizer=sgd,
                        metrics=["acc"])
            print("Finished compiling")
            x_train, y_train = X_train[train],Y_train[train]
            x_val, y_val = X_train[val], Y_train[val]
            train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
            train_dataset = train_dataset.batch(BS)
            val_dataset = tf.data.Dataset.from_tensor_slices((x_val, y_val))
            val_dataset = val_dataset.batch(BS)
            adversarial_training.train(model, train_dataset, val_dataset, epsilons)
            name = model_name+"_"+str(j)+".h5"
            model.save_weights(name)